In [1]:
import spacy
import pandas as pd
import itertools as it
import regex as re
import datetime
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence
import os
import sys
import csv

csv.field_size_limit(sys.maxsize)

131072

In [2]:
data_2010_2017 = pd.read_csv("data_2010_2017_61717.csv",engine='python')

In [12]:
data_2010_2017.sort_values(by = "CIK #")

NameError: name 'data_2010_2017' is not defined

In [15]:
nlp = spacy.load('en')

In [61]:
data_2012 = pd.read_csv("data_file_61717.csv",encoding='utf-8')

In [62]:
data_to_clean = data_2012[["CIK #","Name","company","date","full_text","quarter","report","status","url","year"]]

In [63]:
data_to_clean["first_round"] = data_to_clean["full_text"].str.replace("htm","")

/Users/jasonchiu0803/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [64]:
data_to_clean["first_round"] = data_to_clean["full_text"].str.replace("form|Form|_|AND EXCHANGE COMMISSION|UNITED STATESSECURITIES|Washington, D.C.|10K|10k|Washington, D. C.|Table of Contents|FORM|htm|\x95|\x97|\n|10-K|UNITED STATES SECURITIES AND EXCHANGE COMMISSION","")

/Users/jasonchiu0803/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [65]:
data_to_clean["first_round"]

0      1acme2011.acme2011.   20549   [X]ANNUAL REPORT...
1      1d257108d.     UNITED STATES SECURITIES    205...
2      1fy129282012.FY12  9.28.2012  20549 RANNUAL RE...
3      1mar.MARCH 31, 2012 mar. 20549  (Mark One)RANN...
4      1a2207458z10-k.Use these links to rapidly revi...
5      1unm12312011-10xk.UNM.12.31.2011 -   WASHINGTO...
6      1.STAGE STORES, INC.,  . UNITED STATESSECURITI...
7      1amat10282012-doc.AMAT 10.28.2012- DOCWashingt...
8      1swn123111.SWN 2011  SWN 2011  UNITED STATESSE...
9      1d292916d.     UNITED STATES SECURITIES ANDEXC...
10     1atw-9302012x.  FOR FISCAL YEAR ENDED SEPTEMBE...
11     1d251638d.  2011       UNITED STATES SECURITIE...
12     1d. AVP  2011   20549 ýAnnual Report Pursuant ...
13     1d312481d.  UNITED STATES  SECURITIES   WASHIN...
14     1d281210d.     UNITED STATES SECURITIES ANDEXC...
15     1.BALCHEM CORPORATION  12-31-2011.UNITED STATE...
16     1a12-22331.   UNITED STATESSECURITIES  20549  ...
17     1d259740d.     UNITED ST

In [120]:
def punct_space(token):
    return (token.is_punct or token.is_space or token.is_stop or token.is_digit)

def getting_file_ready(text, process_time = True):
    nospace = re.sub("\n"," ", text)
    new = nlp(nospace)
    return new

def lemmatized_sentence_corpus(data_file):
    parsed_review = getting_file_ready(data_file)
    lemmatized = list()
    for sentence in parsed_review.sents:
        for token in sentence:
            if not punct_space(token):
                lemmatized.append(token.lemma_)
    lemmatized = filter(lambda x: len(x)>2, lemmatized)
    lemmatized = filter(lambda x: "-" not in x, lemmatized)
    return lemmatized

In [ ]:
def punct_space(token):
    """
    helper function to eliminate tokens
    that are pure punctuation, whitespace, stop words, and digits
    """
    
    return token.is_punct or token.is_space or token.is_stop or token.is_digit

def line_review(filename):
    """
    generator function to read in reviews from the file
    and un-escape the original line breaks in the text
    """
    
    with codecs.open(filename, encoding='utf_8') as f:
        for review in f:
            yield review.replace('\\n', '\n')
            
def lemmatized_sentence_corpus(filename):
    """
    generator function to use spaCy to parse reviews,
    lemmatize the text, and yield sentences
    """
    
    for parsed_review in nlp.pipe(line_review(filename),
                                  batch_size=10000, n_threads=4):
        
        for sent in parsed_review.sents:
            yield u' '.join([token.lemma_ for token in sent
                             if not punct_space(token)])

In [75]:
data_to_clean['parsed_text'] = None

In [76]:
# filepaths
unigram_sentences_filepath = os.path.join('unigram.txt')
bigram_sentences_filepath = os.path.join('bigrams.txt')
bigram_model_filepath = os.path.join('bigram_model_all')
trigram_sentences_filepath = os.path.join('trigrams.txt')
trigram_model_filepath = os.path.join('trigram_model_all')
review_txt_filepath = os.path.join('review_text_all.txt')

In [126]:
# load the finished model from disk
import gensim
i = 0 
for row in data_to_clean["full_text_cleaned"]:
    tokens = lemmatized_sentence_corpus(row)
    bigram = Phrases(tokens, min_count=1, threshold=2)
    data_to_clean["parsed_text"][i] = bigram[tokens]
    i = i + 1

/Users/jasonchiu0803/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [129]:
data_to_clean["parsed_text"][0]

[u'1acme2011.acme2011',
 u'x]annual',
 u'report',
 u'pursuant',
 u'section',
 u'security',
 u'exchange',
 u'act',
 u'fiscal',
 u'year',
 u'end',
 u'december',
 u'transition',
 u'report',
 u'pursuant',
 u'section',
 u'ofthe',
 u'security',
 u'exchange',
 u'act',
 u'commission',
 u'file',
 u'number',
 u'07698acme',
 u'united',
 u'corporationexact',
 u'registrant',
 u'specify',
 u'charter',
 u'connecticut06',
 u'0236700(state',
 u'jurisdiction',
 u'of(i.r.s',
 u'employerincorporation',
 u'organization)identification',
 u'round',
 u'hill',
 u'road',
 u'fairfield',
 u'connecticut',
 u'06824(address',
 u'principal',
 u'executive',
 u'offices)(zip',
 u'code',
 u'registrant',
 u'telephone',
 u'number',
 u'include',
 u'area',
 u'code',
 u'security',
 u'register',
 u'pursuant',
 u'section',
 u'12(b',
 u'act',
 u'exchange',
 u'ontitle',
 u'classwhich',
 u'registered$2.50',
 u'par',
 u'value',
 u'common',
 u'stocknyse',
 u'amex',
 u'security',
 u'register',
 u'pursuant',
 u'section',
 u'act',
 u'n

In [ ]:
data_to_clean["trigram"] = None

In [ ]:
i = 0 
for row in data_to_clean["parsed_text"]:
    tokens = lemmatized_sentence_corpus(row)
    trigram = Phrases(bigram[tokens], min_count=1, threshold=2)
    data_to_clean["trigram"][i] = trigram[tokens]
    i = i + 1

In [106]:
from gensim.models import Phrases
documents = ["the mayor of new york was there", "machine learning can be useful sometimes","new york mayor was present"]

sentence_stream = [doc.split(" ") for doc in documents]
bigram = Phrases(sentence_stream, min_count=1, threshold=2)
sent = [u'the', u'mayor', u'of', u'new', u'york', u'was', u'there']
print(bigram[sent])

[u'the', u'mayor', u'of', u'new_york', u'was', u'there']


/Users/jasonchiu0803/anaconda2/lib/python2.7/site-packages/gensim/models/phrases.py:274: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [114]:
trigram = Phrases(bigram[sentence_stream])
sent = [u'the', u'new', u'york', u'times', u'is', u'a', u'newspaper']
print(trigram[bigram[sent]])

[u'12b-2',
 u'12b-2',
 u'10-q',
 u'8-k',
 u'800-sec-0330',
 u'10-q',
 u'8-k',
 u'short-',
 u'holders-0',
 u'-0-total943,000$10.84107,438',
 u'.327-.330',
 u'.336-.359',
 u'13a-15(b',
 u'8-k',
 u'13a-14(a',
 u'15d-14(a',
 u'13a-14(a',
 u'15d-14(a',
 u's-1',
 u's-8',
 u's-8',
 u's-8',
 u's-8',
 u's-8',
 u's-8',
 u'8-k',
 u's-8',
 u'8-k']

In [165]:
data_to_clean["no_number"] = data_to_clean["parsed_text"].\
apply(lambda x: filter(lambda y: not bool(re.search(r'\d',y)), x))

In [179]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()

In [168]:
data_to_clean.columns

Index([            u'CIK #',              u'Name',           u'company',
                    u'date',         u'full_text',           u'quarter',
                  u'report',            u'status',               u'url',
                    u'year',       u'first_round', u'full_text_cleaned',
             u'parsed_text',         u'no_number'],
      dtype='object')

In [169]:
data_to_clean["no_number_sentence"] = data_to_clean["no_number"].\
apply(lambda x:" ".join(x))

In [171]:
X = data_to_clean[["no_number_sentence"]]

In [172]:
y = data_to_clean[["status"]]

In [186]:
X_counts = count_vect.fit_transform(data_to_clean["no_number_sentence"])

In [187]:
X_counts.shape

(627, 231234)

In [192]:
data_to_clean.columns

Index([             u'CIK #',               u'Name',            u'company',
                     u'date',          u'full_text',            u'quarter',
                   u'report',             u'status',                u'url',
                     u'year',        u'first_round',  u'full_text_cleaned',
              u'parsed_text',          u'no_number', u'no_number_sentence'],
      dtype='object')

In [194]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_counts_tfidf = tfidf_transformer.fit_transform(X_counts)

In [195]:
X_counts_tfidf.shape

(627, 231234)

In [301]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_counts_tfidf, y, test_size=0.30, random_state=32)

In [218]:
X_train.shape

(438, 231234)

In [219]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train, y_train)

In [220]:
y_pred = clf.predict(X_test)

In [221]:
y_pred_prob = clf.predict_proba(X_test)

In [222]:
y_pred_prob

array([[  2.17114825e-06,   9.99997829e-01],
       [  1.73672470e-05,   9.99982633e-01],
       [  6.64497365e-06,   9.99993355e-01],
       [  2.22114901e-03,   9.97778851e-01],
       [  2.79639555e-06,   9.99997204e-01],
       [  1.29812212e-06,   9.99998702e-01],
       [  3.95883367e-05,   9.99960412e-01],
       [  3.77709797e-06,   9.99996223e-01],
       [  9.05437229e-07,   9.99999095e-01],
       [  7.95737494e-07,   9.99999204e-01],
       [  3.56642978e-07,   9.99999643e-01],
       [  3.68775248e-08,   9.99999963e-01],
       [  2.06957070e-05,   9.99979304e-01],
       [  3.33612058e-04,   9.99666388e-01],
       [  2.43004501e-03,   9.97569955e-01],
       [  9.89455467e-07,   9.99999011e-01],
       [  1.62934591e-07,   9.99999837e-01],
       [  5.78626202e-07,   9.99999421e-01],
       [  3.71031581e-05,   9.99962897e-01],
       [  1.13328996e-05,   9.99988667e-01],
       [  7.31076445e-07,   9.99999269e-01],
       [  1.71181008e-06,   9.99998288e-01],
       [  

In [223]:
y_train_pred = clf.predict(X_train)

In [224]:
clf.score(X_train, y_train)

0.77168949771689499

In [225]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  0,  47],
       [  0, 142]])

In [227]:
from collections import Counter
list1=list(y_test)
counts = Counter(list1)
print(counts)

Counter({u'status': 1})


In [136]:
print(gnb.score(iris.data, iris.target))

In [236]:
y_test["status"].value_counts()

winner    142
loser      47
Name: status, dtype: int64

# Logistic model

In [237]:
from sklearn import linear_model
logreg=linear_model.LogisticRegression(C=10)

In [241]:
logreg.fit(X_train, y_train)

LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [239]:
result= logreg.predict(X_test)

In [242]:
logreg.score(X_test,y_test)

0.78306878306878303

In [243]:
confusion_matrix(y_test, result)

array([[ 10,  37],
       [  4, 138]])

In [283]:
X_train.shape

(438, 231234)

# Random Forest

In [286]:
from sklearn import ensemble
rf=ensemble.RandomForestClassifier(n_estimators=500, max_depth=4, min_samples_leaf=2, max_features=2312)
rf=rf.fit(X_train, y_train)
print rf.score(X_train, y_train)
print rf.score(X_test, y_test)

/Users/jasonchiu0803/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  app.launch_new_instance()


0.87899543379
0.746031746032


In [290]:
tree_y = rf.predict(X_test)

In [294]:
confusion_matrix(y_test, tree_y)

array([[  1,  46],
       [  2, 140]])

In [302]:
y_train["status"] == "loser"

10      True
66     False
334    False
611    False
225    False
405    False
52     False
137    False
411     True
526     True
300    False
13     False
101    False
33     False
533     True
391     True
323    False
122    False
197    False
417    False
12      True
269     True
146    False
192    False
201    False
388    False
72     False
506    False
344     True
464    False
       ...  
221    False
564     True
474    False
120     True
395    False
62     False
589    False
166    False
249    False
185    False
100    False
434    False
594    False
554    False
35     False
418     True
577    False
259    False
465     True
4      False
9      False
234    False
601    False
510     True
71     False
252    False
403    False
88     False
310     True
555     True
Name: status, Length: 438, dtype: bool

# SVM

In [292]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
parameters = {"kernel":("linear", "rbf"), "C":[1, 10]}
svr = svm.SVC()
clf_2 = GridSearchCV(svr, parameters,cv=3)

In [304]:
a = clf_2.fit(X_train, y_train["status"] == "loser")

In [305]:
b = a.best_estimator_

In [306]:
a.best_params_

{'C': 10, 'kernel': 'linear'}

In [307]:
b.fit(X_train, y_train)

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [313]:
svm_y = b.predict(X_test)

In [315]:
confusion_matrix(y_test, svm_y)

array([[ 12,  35],
       [  6, 136]])

In [316]:
b.score(X_test,y_test)

0.78306878306878303

In [317]:
svm_y

array([u'winner', u'winner', u'winner', u'winner', u'winner', u'winner',
       u'winner', u'winner', u'winner', u'winner', u'winner', u'winner',
       u'winner', u'winner', u'winner', u'winner', u'winner', u'winner',
       u'winner', u'winner', u'winner', u'winner', u'loser', u'winner',
       u'winner', u'winner', u'winner', u'winner', u'winner', u'winner',
       u'winner', u'winner', u'loser', u'winner', u'winner', u'loser',
       u'winner', u'winner', u'winner', u'loser', u'winner', u'winner',
       u'winner', u'winner', u'winner', u'winner', u'winner', u'winner',
       u'winner', u'winner', u'winner', u'winner', u'winner', u'winner',
       u'loser', u'loser', u'winner', u'winner', u'winner', u'winner',
       u'winner', u'loser', u'loser', u'winner', u'winner', u'winner',
       u'winner', u'winner', u'winner', u'winner', u'winner', u'loser',
       u'winner', u'winner', u'winner', u'winner', u'winner', u'loser',
       u'winner', u'winner', u'winner', u'winner', u'winner',